In [5]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import sqlite3
from sklearn import preprocessing

In [6]:
pd.options.display.max_columns = None
players = pd.read_csv('players_mejor_3.csv')

In [7]:
players = players[players["best_of"] == 3]

In [8]:
#borramos las columnas correspondientes a variables que se encuentran desagregadas por mapa y las de resumen del partido total
players.drop(columns=['m1_kills', 'm1_assists', 'm1_deaths', 'm2_kills', 'm2_assists', 'm2_deaths', 'm3_kills', 'm3_assists', 'm3_deaths', 'kills', 'assists', 'deaths', 'hs', 'flash_assists', 'kast', 'adr', 'fkdiff', 'rating','kills_ct', 'deaths_ct', 'kddiff_ct', 'adr_ct','kast_ct','rating_ct','kills_t','deaths_t','kddiff_t','adr_t','kast_t','rating_t', 'kddiff','m1_kills_ct','m1_deaths_ct','m1_kddiff_ct','m1_adr_ct','m1_kast_ct','m1_rating_ct','m1_kills_t','m1_deaths_t','m1_kddiff_t','m1_adr_t','m1_kast_t','m1_rating_t','m2_kills_ct','m2_deaths_ct','m2_kddiff_ct','m2_adr_ct','m2_kast_ct','m2_rating_ct','m2_kills_t','m2_deaths_t','m2_kddiff_t','m2_adr_t','m2_kast_t','m2_rating_t','m3_kills_ct','m3_deaths_ct','m3_kddiff_ct','m3_adr_ct','m3_kast_ct','m3_rating_ct','m3_kills_t','m3_deaths_t','m3_kddiff_t','m3_kast_t','m3_rating_t','m3_adr_t'],axis=1,inplace=True)

In [9]:
players.sample(3)

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,m3_adr,m3_fkdiff,m3_rating
187555,2016-02-05,cajunb,Astralis,G2,Denmark,2469,2300594,2013,Game Show Global eSports Cup 2016 Finals,3,Train,Overpass,Cache,8,NaN,81.5,10,83.1,-2,1.35,8.0,NaN,69.0,-11.0,46.7,-3.0,0.75,8.0,0.0,70.6,-1.0,81.6,2.0,1.08
33575,2019-06-21,mouz,Aristocracy,Chaos,Poland,8323,2334493,4605,UCC Summer Smash,3,Overpass,Train,NaN,5,1.0,57.1,-8,45.8,-2,0.63,3.0,1.0,55.2,-12.0,46.2,-3.0,0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114541,2017-11-17,ofnu,Tainted Minds,Grayhound,New Zealand,8767,2317371,3166,ZEN League Season 2 Finals,3,Cache,Inferno,Overpass,13,4.0,65.5,5,82.0,-2,1.09,11.0,1.0,79.3,8.0,59.5,-1.0,1.21,8.0,3.0,70.0,-5.0,60.7,-1.0,0.82


In [10]:
players.shape

(178728, 34)

In [11]:
# Crear una conexión a una base de datos en memoria
conn = sqlite3.connect(":memory:")

# Guardar los datos en una tabla temporal
players.to_sql("players", conn, if_exists="replace")

# Ejecutar la consulta SQL para filtrar los resultados
# al armar el dataset nuevo descartamos las columnas que son ids (menos match_id que lo usamos despues para transformar team y opponent), date, player_name y event_name al no utilizarlas
# tambien borramos las columnas correspondientes a valores futuros al momento de predecir por lo que no debemos mantener en entre los datos
query = """
SELECT match_id, team, opponent, country, map_1 AS first_map, m1_hs AS first_map_hs, m1_kast AS first_map_kast, m1_kddiff AS first_map_kddiff, m1_adr AS first_map_adr,
    m1_fkdiff AS first_map_fkdiff, m1_rating AS first_map_rating, map_2 AS second_map, m2_kddiff AS second_map_kddiff
FROM players;
"""

#aca lo que hacemos es para no perder los registros que solo se jugaron dos mapas porque los ganaron de corrido
#si el map3 tiene NaN no creamos el segundo registro
query2 = """
SELECT match_id, team, opponent, country, map_2 AS first_map, m2_hs AS first_map_hs, m2_kast AS first_map_kast, m2_kddiff AS first_map_kddiff, m2_adr AS first_map_adr,
    m2_fkdiff AS first_map_fkdiff, m2_rating AS first_map_rating, map_3 AS second_map, m3_kddiff AS second_map_kddiff
FROM players
WHERE map_3 IS NOT NULL;
"""

# Obtener un dataset con los registros de la primer query
players_nuevo_first = pd.read_sql_query(query, conn)
# Obterner un dataset con los registros de la segunda query
players_nuevo_second = pd.read_sql_query(query2, conn)

conn.close()

# Concatenamos ambos dataset para obtener el dataset final
players_nuevo = pd.concat([players_nuevo_first, players_nuevo_second], ignore_index=True)


In [12]:
players_nuevo.sample(3)

,match_id,team,opponent,country,first_map,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map,second_map_kddiff
56475,2329807,Red Reserve,DreamEaters,Sweden,Cache,6.0,58.6,-1.0,66.0,-3.0,0.93,Overpass,14.0
42655,2332280,Sprout,Epsilon,Germany,Inferno,5.0,75.0,-3.0,54.6,0.0,1.00,Mirage,1.0
44054,2332411,Valiance,Windigo,Serbia,Nuke,7.0,61.9,-4.0,76.4,2.0,0.98,Dust2,10.0


In [13]:
players_nuevo.shape

(244370, 13)

In [14]:
num_country = players_nuevo['country'].nunique()
num_first_map = players_nuevo['first_map'].nunique()
num_second_map = players_nuevo['second_map'].nunique()

# Imprimir la cantidad de valores distintos
print("Cantidad de valores distintos en la columna 'country':", num_country)
print("Cantidad de valores distintos en la columna 'first_map':", num_first_map)
print("Cantidad de valores distintos en la columna 'second_map':", num_second_map)

Cantidad de valores distintos en la columna 'country': 109
Cantidad de valores distintos en la columna 'first_map': 10
Cantidad de valores distintos en la columna 'second_map': 10


In [15]:
# hacer one hot enconder a country, first_map y second_map:
# creamos las columnas encodeadas y las formateamos
players_nuevo = pd.get_dummies(players_nuevo, columns=['country','first_map', 'second_map'])

players_nuevo.columns = players_nuevo.columns.str.lower()
BETTER_COLUMN_NAMES = {
    'country_bosnia and herzegovina': 'country_bosnia_and_herzegovina',
    'country_costa rica': 'country_costa_rica',
    'country_czech republic': 'country_czech_republic',
    'country_dominican republic': 'country_dominican_republic',
    'country_el salvador': 'country_el_salvador',
    'country_faroe islands': 'country_faroe_islands',
    'country_hong kong': 'country_hong_kong',
    'country_new zealand': 'country_new_zealand',
    'country_saudi arabia': 'country_saudi_arabia',
    'country_south africa': 'country_south_africa',
    'country_sri lanka': 'country_sri_lanka',
    'country_united arab emirates': 'country_united_arab_emirates',
    'country_united kingdom': 'country_united_kingdom',
    'country_united states': 'country_united_states'
}
players_nuevo.rename(columns=BETTER_COLUMN_NAMES, inplace=True)


In [16]:
players_nuevo.shape

(244370, 139)

In [17]:
# codificacion de variables team y opponent como el puesto que ocupaba cada equipo en el ranking de cs oficial en el dia del partido
# para esto utilizamos el dataset results.csv del mismo lugar donde descargamos el dataset que estamos utilizando
# lo que hacemos es por cada match_id, reemplazar las columnas team y opponent por el numero de ranking (columnas team_ranking y opponent_ranking)

In [18]:
# este dataset contiene tres registros por cada partido (el resultado de cada mapa) por lo que lo leemos y nos quedamos con cualquier de esos tres registros, ya que solo nos interesa el ranking y son los mismos valores en los tres
results = pd.read_csv('results.csv').drop_duplicates(subset='match_id', keep='first')

# vemos un ejemplo de un registro de results
results.sample(1)

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
44954,2016-03-03,Virtus.pro,Astralis,Overpass,6,16,2,1,5,10,1,6,2036,2301162,7,4,6,16,2


In [19]:
# creamos las columnas nuevas que van a alojar el valor de ranking
players_nuevo["team_ranking"] = -1
players_nuevo["opponent_ranking"] = -1

#recorremos players_nuevo
for index, registro in players_nuevo.iterrows():
    #buscamos el registro de results que coincida con el actual match_id del registro de players_nuevo
    row = results.loc[results['match_id'] == registro['match_id']]
    #si encontramos el match en results
    if len(row) > 0:
        #asignamos las nuevas columnas del ranking de cada equipo dependiendo como venian ordenadas
        if registro['team'] == row['team_1'].values[0] and registro['opponent'] == row['team_2'].values[0]:
            players_nuevo.loc[index, 'team_ranking'] = row['rank_1'].values[0]
            players_nuevo.loc[index, 'opponent_ranking'] = row['rank_2'].values[0]
        elif registro['opponent'] == row['team_1'].values[0] and registro['team'] == row['team_2'].values[0]:
            players_nuevo.loc[index, 'team_ranking'] = row['rank_2'].values[0]
            players_nuevo.loc[index, 'opponent_ranking'] = row['rank_1'].values[0]

players_nuevo.sample(3)

,match_id,team,opponent,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
67416,2326941,Monolith,Evolution,11.0,77.8,6.0,75.2,0.0,1.27,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,101,161
234935,2304551,Space Soldiers,PRIDE,17.0,83.3,14.0,134.7,1.0,1.91,11.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,

In [20]:
#borramos los registros de players_nuevo que quedan con columnas nuevas ya que no tenian ocurrencia en results, perdiendo datos
indices_a_eliminar = []
for index, registro in players_nuevo.iterrows():
    # Verifica si las columnas seleccionadas contienen el valor -1
    if registro['team_ranking'] == -1 or registro['opponent_ranking'] == -1:
        # guardamos el indice para eliminarlo despues
        indices_a_eliminar.append(index)

# Elimina los registros en un solo paso
players_nuevo.drop(indices_a_eliminar, inplace=True)

#borramos tambien las columnas match_id, team y opponent que ya no utilizamos
players_nuevo.drop(columns=['team', 'opponent', 'match_id'],axis=1,inplace=True)
        
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
228855,16.0,64.3,0.0,83.6,-2.0,1.02,5.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,33,37
223360,7.0,77.3,1.0,87.9,3.0,1.13,8.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,46,40
48892,2.0,66.7,-13.0,44.

In [21]:
players_nuevo.shape

(165946, 138)

In [22]:
# normalizamos todas las columnas necesarias
columnas_a_normalizar = ['first_map_hs','first_map_kast', 'first_map_kddiff', 'first_map_adr', 'first_map_fkdiff', 'team_ranking', 'opponent_ranking']

# Crear un objeto MinMaxScaler para normalizar las columnas
scaler = preprocessing.MinMaxScaler()

# Normalizar las columnas seleccionadas
players_nuevo[columnas_a_normalizar] = scaler.fit_transform(players_nuevo[columnas_a_normalizar])



In [23]:
#borramos los registros de players_nuevo que quedan con valores nulos en first_map_kddiff y second_map_kddiff (siguen teniendo -1 como al inicializar)
indices_a_eliminar = []
for index, registro in players_nuevo.iterrows():
    # Verifica si las columnas seleccionadas contienen el valor -1
    if registro.isnull().any():
        # guardamos el indice para eliminarlo despues
        indices_a_eliminar.append(index)

# Elimina los registros en un solo paso
players_nuevo.drop(indices_a_eliminar, inplace=True)

In [24]:
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
96731,0.078947,0.512936,0.200000,0.261812,0.40,0.61,3.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,0.031414,0.041885
128391,0.289474,0.750281,0.566667,0.518066,0.44,1.43,11.0,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,

In [30]:
# por ultimo, convertimos second_map_kddiff en las etiquetas 1 o 0 dependiendo si son positivas o negativas, respectivamente
players_nuevo['second_map_kddiff'] = np.where(players_nuevo['second_map_kddiff'] >= 0, 1, 0)

players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
948,0.157895,0.573678,0.366667,0.312396,0.44,0.74,0,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,0.342932,0.170157
105581,0.368421,0.550056,0.433333,0.472485,0.40,1.05,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,F

In [29]:
#chequeamos valores nulos
players_nuevo.isnull().sum()

first_map_hs           0
first_map_kast         0
first_map_kddiff       0
first_map_adr          0
first_map_fkdiff       0
                      ..
second_map_overpass    0
second_map_train       0
second_map_vertigo     0
team_ranking           0
opponent_ranking       0
Length: 138, dtype: int64

In [32]:
#HASTA ACA TENEMOS LISTO EL PREPROCESAMIENTO

#players_nuevo.to_csv('players_preprocesado.csv', index=False)

players_nuevo = pd.read_csv('players_preprocesado.csv')
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
98358,0.184211,0.847019,0.566667,0.409672,0.48,1.37,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,0.151832,0.376963
17752,0.368421,0.670416,0.550000,0.478043,0.44,1.24,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,

In [33]:
#SI QUEREMOS SIN COUNTRIES
#prefix = 'country_'
#columns_to_drop = [col for col in players_nuevo.columns if col.startswith(prefix)]
#players_nuevo_sin_paises = players_nuevo.drop(columns=columns_to_drop)
#players_nuevo_sin_paises.to_csv('players_preprocesado_sin_paises.csv', index=False)

players_nuevo_sin_paises = pd.read_csv('players_preprocesado_sin_paises.csv')
players_nuevo_sin_paises.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
49249,0.184211,0.767154,0.500000,0.545859,0.60,1.45,1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,0.259162,0.141361
57793,0.157895,0.897638,0.583333,0.448027,0.44,1.46,1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0.353403,0.408377
81839,0.052632,0.689539,0.366667,0.391329,0.48,1.01,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0.133508,0.238220


## Definición de métrica:

Elegimos ACCURACY (precisión).
La precisión es la proporción de predicciones correctas realizadas por el modelo. Se calcula dividiendo el número de predicciones correctas entre el número total de predicciones.
Elegimos esta métrica ya que es especialmente util en clasificación y además es fácil de interpretar.
Su desventaja principal se encuentra en problemas cuyas clases en el conjunto de datos están desbalanceadas, cosa que no presentará inconvenientes en nuestro dataset (valor obtenido en tp1 -> kddiff positivo = 50,07% / kddiff negativo = 49,03%)

## División del dataset:

In [39]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en variable de entrada (entradas) y variable objetivo (salida)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(players_nuevo, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train_y = train['second_map_kddiff']
train_x = train.drop(columns=['second_map_kddiff'])

validation_y = validation['second_map_kddiff']
validation_x = validation.drop(columns=['second_map_kddiff'])

test_y = test['second_map_kddiff']
test_x = test.drop(columns=['second_map_kddiff'])

# Mostramos las dimensiones finales de cada porcion del dataset
train_x.shape, train_y.shape, validation_x.shape, validation_y.shape, test_x.shape, test_y.shape




((98353, 137), (98353,), (32784, 137), (32784,), (32785, 137), (32785,))

## Feature engineering del dataset

## Entrenamiento de modelos:

### Regresion logística

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

regression_model = LogisticRegression(random_state=42, max_iter=1000)

regression_model.fit(train_x, train_y)

y_pred = regression_model.predict(test_x)

print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

#evaluate_model(regression_model, title='Logistic Regression')

Accuracy on test: 0.5854506634131462


### Random forest

In [41]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier()

forest_model.fit(train_x, train_y)

y_pred = forest_model.predict(test_x)

print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

#evaluate_model(forest_model, title='Random Forest')

Accuracy on test: 0.583102028366631


### K nearest neighbors (KNN)

In [42]:
from sklearn.neighbors import KNeighborsClassifier

#Parametros varios:
K = 5

kkn_model = KNeighborsClassifier(n_neighbors=K)

kkn_model.fit(train_x, train_y)

y_pred = kkn_model.predict(test_x)

print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

#evaluate_model(kkn_model, title='K nearest neighbors (KNN)')

Accuracy on test: 0.5453408570992833


### Dummie Classifier

In [270]:
from sklearn.dummy import DummyClassifier

# Crear el clasificador dummy
dummy_clf = DummyClassifier(strategy='uniform')

# Ajustar el clasificador dummy a tus datos
dummy_clf.fit(train_x, train_y)

# Realizar predicciones con el clasificador dummy
y_pred = dummy_clf.predict(test_x)


print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

Accuracy on train: 0.5011895804949468


## Definición de hiperparametros implementando GridSearch: